In [23]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import plotly.express as px

In [24]:
# load data from previous notebooks
aadt_df = pd.read_csv('intermediate/external-historic-aadt.csv')
linear_forecast_df = pd.read_csv('results/linear-forecasts.csv')
externals_df = pd.read_csv('params/externals.csv')
previous_forecasts_df = pd.read_csv('intermediate/previous-forecasts.csv')

Create interactive chart with assist from ChatGPT: https://chat.openai.com/share/496c1584-22b6-4c72-9815-395e11718907

In [27]:
# MAKE INTERACTIVE CHART

# Create a lookup dictionary
name_lookup = externals_df.set_index('externalid')['name'].to_dict()

# Map names to the forecast df
linear_forecast_df['name'] = linear_forecast_df['externalid'].map(name_lookup)

# Build the final string
linear_forecast_df['external'] = (
    'Ext # ' +
    linear_forecast_df['externalid'].fillna(0).astype(int).astype(str) +
    ' - ' +
    linear_forecast_df['name'].fillna('')
)

# Map names to the forecast df
previous_forecasts_df['name'] = previous_forecasts_df['externalid'].map(name_lookup)

# Build the final string
previous_forecasts_df['external'] = (
    'Ext # ' +
    previous_forecasts_df['externalid'].fillna(0).astype(int).astype(str) +
    ' - ' +
    previous_forecasts_df['name'].fillna('')
)

# Map names to the forecast df
aadt_df['name'] = aadt_df['externalid'].map(name_lookup)

# Build the final string
aadt_df['external'] = (
    'Ext # ' +
    aadt_df['externalid'].fillna(0).astype(int).astype(str) +
    ' - ' +
    aadt_df['name'].fillna('')
)

# Create a dropdown widget for external selection
external_widget = widgets.Dropdown(
    options=linear_forecast_df['external'].unique().tolist(),
    description='external:',
    disabled=False
)

projgrp_widget = widgets.SelectMultiple(
    options=linear_forecast_df['PROJ_GRP'].unique(),
    description='Proj Groups:',
    disabled=False
)

# Display function
def plot_data(external, projgrp):

    output.clear_output()
    global firstTime
   
    if firstTime:
        firstTime = False
    else:
        filteredlinear_forecast_df = linear_forecast_df[(linear_forecast_df['external'] == external) & (linear_forecast_df['PROJ_GRP'].isin(projgrp))]
        filteredaadt_df = aadt_df[(aadt_df['external'] == external)]
        filteredprevious_forecast_df = previous_forecasts_df[(previous_forecasts_df['external'] == external)]
        fig = px.line(filteredlinear_forecast_df, x='year', y='LIN_FORECAST', color='PROJ_GRP',
                    title=f'Linear Forecast for {external}',
                    labels={'LIN_FORECAST': 'AADT', 'year': 'Year'},
                    markers=True) 
        fig.update_traces(line=dict(dash='dash'))
        fig.update_traces(marker=dict(size=20))
    
        # Adding the scatter plot from dfHistoricAADT
        fig.add_scatter(x=filteredaadt_df['year'], y=filteredaadt_df['AADT'], mode='markers', 
                        marker=dict(size=10, color='darkgrey', symbol='circle'), name='Historic AADT')
        
        fig.update_layout(height=500)
        fig.update_xaxes(range=[1980, 2055+2])

        max_LinF = linear_forecast_df[(linear_forecast_df['external'] == external)]['LIN_FORECAST'].max()
        max_Aadt = filteredaadt_df['AADT'].max()
        overall_max = max(max_LinF, max_Aadt)
        fig.update_yaxes(range=[0,overall_max+1000])
    
        # Adding the scatter plot from dfHistoricAADT
        fig.add_scatter(x=filteredprevious_forecast_df['year'], y=filteredprevious_forecast_df['AADT'], mode='markers', 
                        marker=dict(size=10, color='orange', symbol='circle'), name='v9.0 Forecast')
                                    
        fig.show()
        
# to keep VS CODE from duplicating chart images use global firstTime
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([external_widget, projgrp_widget])

# create interactive widget
interactive_output = widgets.interactive_output(plot_data, {'external': external_widget, 'projgrp': projgrp_widget})

# Display a markdown heading
display(hbox)
display(interactive_output)
display(output)
    

Output()

Output()

In [ ]:
selected_forecast_df = pd.DataFrame([
    [3601, 'Since 2003'],
    [3602, 'Since 2002'],
    [3603, 'Since 2016'],
    [3604, 'Since 2011'],
    [3605, 'Since 2021'],
    [3606, 'Since 2011'],
    [3607, 'Since 2007'],
    [3608, 'Since 2014'],
    [3609, 'Since 2003'],
    [3610, ''],
    [3611, ''],
    [3612, 'Since 2009'],
    [3613, 'Since 2009'],
    [3614, 'Since 2006'],
    [3615, 'Since 2007'],
    [3616, 'Since 2011'],
    [3617, 'Since 2011'],
    [3618, 'Since 2007'],
    [3619, ''],
    [3620, 'Since 2006'],
    [3621, ''],
    [3622, ''],
    [3623, 'Since 2004'],
    [3624, 'Since 2017'],
    [3625, ''],
    [3626, 'Since 2007'],
    [3627, ''],
    [3628, 'Since 2022'],
    [3629, 'Since 2011']
], columns=['externalid','PROJ_GRP'])

linear_forecast_selected_df = pd.merge(selected_forecast_df, linear_forecast_df, on=['externalid','PROJ_GRP'])
linear_forecast_selected_df

,externalid,PROJ_GRP,segid,year,LIN_FORECAST,name,external
0,3601,Since 2003,1082_000.0,2003,412,FAR-1082 Bird Refuge,Ext # 3601 - FAR-1082 Bird Refuge
1,3601,Since 2003,1082_000.0,2027,890,FAR-1082 Bird Refuge,Ext # 3601 - FAR-1082 Bird Refuge
2,3601,Since 2003,1082_000.0,2032,989,FAR-1082 Bird Refuge,Ext # 3601 - FAR-1082 Bird Refuge
3,3601,Since 2003,1082_000.0,2036,1069,FAR-1082 Bird Refuge,Ext # 3601 - FAR-1082 Bird Refuge
4,3601,Since 2003,1082_000.0,2046,1268,FAR-1082 Bird Refuge,Ext # 3601 - FAR-1082 Bird Refuge
...,...,...,...,...,...,...,...
133,3629,Since 2011,1826_004.9,2027,3823,FAR-1826 South Ridge Farms,Ext # 3629 - FAR-1826 South Ridge Farms
134,3629,Since 2011,1826_004.9,2032,4248,FAR-1826 South Ridge Farms,Ext # 3629 - FAR-1826 South Ridge Farms
135,3629,Since 2011,1826_004.9,2036,4588,FAR-1826 South Ridge Farms,Ext # 3629 - FAR-1826 South Ridge Farms
136,3629,Since 2011,1826_004.9,2046,5437,FAR-1826 South Ridge Farms,Ext # 3629 - FAR-1826 South Ridge Farms


In [28]:
# MAKE INTERACTIVE CHART

# Create a dropdown widget for external selection
external_widget = widgets.Dropdown(
    options=linear_forecast_selected_df['external'].unique().tolist(),
    description='external:',
    disabled=False
)

# Display function
def plot_data(external):

    output.clear_output()
    global firstTime
   
    if firstTime:
        firstTime = False
    else:
        filteredlinear_forecast_selected_df = linear_forecast_selected_df[(linear_forecast_selected_df['external'] == external)]
        filteredaadt_df = aadt_df[(aadt_df['external'] == external)]
        filteredprevious_forecast_df = previous_forecasts_df[(previous_forecasts_df['external'] == external)]
        fig = px.line(filteredlinear_forecast_selected_df, x='year', y='LIN_FORECAST', color='PROJ_GRP',
                    title=f'Linear Forecast for {external}',
                    labels={'LIN_FORECAST': 'AADT', 'year': 'Year'},
                    markers=True) 
        fig.update_traces(line=dict(dash='dash'))
        fig.update_traces(marker=dict(size=20))
    
        # Adding the scatter plot from dfHistoricAADT
        fig.add_scatter(x=filteredaadt_df['year'], y=filteredaadt_df['AADT'], mode='markers', 
                        marker=dict(size=10, color='darkgrey', symbol='circle'), name='Historic AADT')
        
        fig.update_layout(height=500)
        fig.update_xaxes(range=[1980, 2055+2])
    
        # Adding the scatter plot from dfHistoricAADT
        fig.add_scatter(x=filteredprevious_forecast_df['year'], y=filteredprevious_forecast_df['AADT'], mode='markers', 
                        marker=dict(size=10, color='orange', symbol='circle'), name='v9.0 Forecast')

        max_LinF = linear_forecast_selected_df[(linear_forecast_selected_df['external'] == external)]['LIN_FORECAST'].max()
        max_Aadt = filteredaadt_df['AADT'].max()
        overall_max = max(max_LinF, max_Aadt)
        fig.update_yaxes(range=[0,overall_max+1000])

        fig.show()

In [29]:
manual_adj_df = pd.DataFrame([
    [3616, 20]
])

# to keep VS CODE from duplicating chart images use global firstTime
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([external_widget])

# create interactive widget
interactive_output = widgets.interactive_output(plot_data, {'external': external_widget})

# Display a markdown heading
display(hbox)
display(interactive_output)
display(output)

Output()

Output()